<a href="https://colab.research.google.com/github/jmhuer/utaustin_optimization/blob/main/homework6/homework6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 6
In this problem set you will implement SGD and SVRG and compare the two to each other, and also to GD.

In [1]:
!pip install gdown
! git clone https://github.com/jmhuer/optimization_tools
from optimization_tools.utils import download_gdrive

# we pass the ID between two slashes '/{ID}/'  
digits = '1X4fvOpQ2LK81-D3Fw4Oxugdvgm1dDJ3d'
news = '1Sc6ew-Ti8uNAPScD9P1UXPXrziPqaEwd'
download_gdrive(digits)
download_gdrive(news)


Cloning into 'optimization_tools'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 46 (delta 12), reused 39 (delta 8), pack-reused 0
Unpacking objects: 100% (46/46), done.
Downloading...
From: https://drive.google.com/uc?id=1X4fvOpQ2LK81-D3Fw4Oxugdvgm1dDJ3d
To: /content/digits.zip
100%|##########| 98.7k/98.7k [00:00<00:00, 35.7MB/s]

Downloading...
From: https://drive.google.com/uc?id=1Sc6ew-Ti8uNAPScD9P1UXPXrziPqaEwd
To: /content/news.zip
8.53MB [00:00, 74.8MB/s]



# Problem 1: Stochastic Variance Reduced Gradient Descent (SVRG)

As we discussed in the video lectures, decomposable functions of the form
$$
\min_{\omega} \left [ F(\omega) = \frac{1}{n} \sum_i^n f_i(\omega) \right ],
$$
are very common in statistics/ML problems. Here, each $f_i$ corresponds to a loss for a particular training example. For
example, if $f_i(\omega) = (\omega^\top x_i - y_i)^2$, then $F(\omega)$ is a least
squares regression problem. The standard gradient descent (GD) update 
$$
\omega_t = \omega_{t-1} - \eta_t \nabla F(\omega_{t-1})
$$

evaluates the full gradient $\nabla F(\omega) = \frac{1}{n} \sum_i^n \nabla
f_i(\omega)$, which requires evaluating $n$ derivatives. This can be
prohibitively expensive when the number of training examples $n$ is large. SGD evaluates
the gradient of one (or a small subset) of the training examples--drawn
randomly from ${1,...n}$--per iteration:
$$
\omega_t = \omega_{t-1} - \eta_t \nabla f_i(\omega_{t-1}).
$$

In expectation, the updates are equivalent, but SGD has the computational
advantage of only evaluating a single gradient $\nabla f_i(\omega)$. The
disadvatage is that the randomness introduces variance, which slows
convergence. This was our motivation in class to introduce the SVRG algorithm.

Given the dataset in **digits.zip**, plot the performance of GD, SGD, and SVRG for logistic regression with $l2$ regularization in terms of negative log likelihood on the training data against the number of gradient evaluations for a single training example (GD performs $n$ such evaluations per iteration and SGD performs $1$). Choose the $l2$ parameter to optimize performance on the test set. How does the choice of $T$ (the number of inner loops) affect the performance of SVRG? There should be one plot with a title and three lines with different colors, markers, and legend labels.



In [10]:
import zipfile as zipfile
import pandas as pd
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import time
import pdb
from tqdm import tqdm

%matplotlib inline

#sample code to load digits.zip
def loaddata(filename):
    data={}
    with zipfile.ZipFile(filename) as z:
        for filename in z.namelist():
          data[filename] = pd.read_csv(z.open(filename), sep=' ', header=None)
    return data

digits_dict = loaddata('./digits.zip')
print(digits_dict.keys())
X_digits_train = digits_dict['X_digits_train.csv']
X_digits_test = digits_dict['X_digits_test.csv']
y_digits_train = digits_dict['y_digits_train.csv'].to_numpy(dtype=int).ravel()
y_digits_test = digits_dict['y_digits_test.csv'].to_numpy(dtype=int).ravel()

dict_keys(['X_digits_test.csv', 'X_digits_train.csv', 'y_digits_test.csv', 'y_digits_train.csv'])


In [47]:
import numpy as np
import numpy.random as rn
import numpy.linalg as la
import matplotlib.pyplot as plt
import time
import plotly.graph_objects as graph
import numpy.linalg as la
from tqdm import tqdm

class MSE:
    def __init__(self, train_data):
        A ,b = train_data['data'], train_data['labels']
        self.loss = lambda weight : (1./b.shape[0])*(.5)*np.sum((np.dot(A,weight)-b)**2)
        self.grad_loss = lambda weight : (1./b.shape[0])*np.dot(A.T,np.dot(A,weight)-b)
    def eval(self, weight):
        return self.loss(weight) 
    def gradient(self,weight):
        return self.grad_loss(weight) 
    def test_loss(self, test_data, weights):
        A ,b = test_data['data'], test_data['labels']
        test_loss = lambda weights : (1./b.shape[0])*(.5)*np.sum((np.dot(A,weights)-b)**2)
        return test_loss(weights) 


class SVRG:
    def __init__(self, A, b, lmda = 1e-3):
        self.A = A
        self.b = b
        self.lmda = lmda
        self.loss = lambda weight : (1./b.shape[0])*(.5)*np.sum((np.dot(A,weight)-b)**2)
        self.reg  = lambda weight : self.lmda*la.norm(weight,1)
        self.grad_loss = lambda weight : (1./b.shape[0])*np.dot(A.T,np.dot(A,weight)-b)
        self.grad_reg  = lambda weight : self.lmda*la.norm(weight,1) 
    def eval(self, weight):
        return self.loss(weight) 
    def gradient(self,weight):
        return self.grad_loss(weight) 

  

In [50]:
class GD:
    def __init__(self, function, x_init, epochs, learning_rate):
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.history = {"step": [],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [self.x_init], 
                        "opt_dif": [],
                        "test_loss": []}
    def go(self, test_set = None):
        for i in tqdm(range(self.epochs)):
          ##RETRIVE OLD VALS
          x_old = self.history['x_vals'][-1]
          ##EVALUATE HERE
          y = self.function.eval(x_old)
          g = self.function.gradient(x_old)
          ##UPDATE HERE 
          x = x_old - self.learning_rate(i) * g
          ##STORE HISTORY
          self.history['step'].append(i)
          self.history['function_vals'].append(float(y))
          self.history['grad_vals'].append(g)
          self.history['x_vals'].append(x)
          if test_set:  
              self.history['test_loss'].append(float(self.function.test_loss(test_data = test_set, weights = x)))

In [51]:
def L_M(A):
  sm , sc = (1/A.shape[0])*la.norm(A,2)**2,(1/A.shape[0])*la.norm(A,-2)**2
  return sm , sc

train_data = {"data"  : X_digits_train,
              "labels": y_digits_train}

test_data  = {"data"  : X_digits_test,
              "labels": y_digits_test}

sm , sc = L_M(X_digits_train)

gd    = GD(MSE(train_data), x_init = np.random.rand(X_digits_train.shape[1])*0, epochs = int(5e2), learning_rate = lambda dummy : 1/sm)

gd.go(test_set=test_data)

# plot the train loss
all_history = { "GD"  : gd.history }

fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="GD")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["function_vals"],
                                name = i))
fig.show()



# plot the test loss
fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="GD")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["test_loss"],
                                name = i))
fig.show()

100%|██████████| 500/500 [00:00<00:00, 1036.40it/s]


# Problem 2: Newsgroup Dataset Optimization

Using any approach, optimize performance of logistic regression on the test set in **news.zip** and compare the performance of your approach to standard SGD. This dataset is the full-dimensional newsgroup dataset (as opposed to the compressed version you worked with previously). The $X$ matrices are stored in sparse matrix format and can be read using scipy.io.mmread. As the dataset is large and high-dimensional, you will have to decide on how best to allocate your computational resources. Try to utilize the sparsity of the data (i.e., don't just convert it to a dense matrix and spend all your time multiplying zeros). You may use any of the techniques covered in class or ideas from outside class (e.g., momentum, variance reduction, minibatches, adaptive learning rates, preprocessing). Describe your methodology and comment on what you found improved performance and why. Plot the performance (negative log likelihood) of your method against standard SGD in terms of the number of gradient evaluations. 

In [ ]:
from scipy.io import mmread
import sklearn.feature_selection


#sample code to load news.zip
def loadnewsdata(filename='./news.zip'):
    data={}
    with zipfile.ZipFile(filename) as z:
        for filename in z.namelist():
          if 'csv' in filename:
            data[filename] = pd.read_csv(z.open(filename), sep=' ', header=None)
          elif 'mtx' in filename:
            data[filename] = mmread(z.open(filename))
          else:
            raise Exception('unexpected filetype') 
    return data

news_dict = loadnewsdata('./news.zip')
print(news_dict.keys())
X_news_train = news_dict['X_news_train.mtx']
X_news_test = news_dict['X_news_test.mtx']
y_news_train = news_dict['y_news_train.csv'].to_numpy(dtype=int).ravel()
y_news_test = news_dict['y_news_test.csv'].to_numpy(dtype=int).ravel()

dict_keys(['X_news_test.mtx', 'X_news_train.mtx', 'y_news_test.csv', 'y_news_train.csv'])
